# 專案：應用於臺北捷運時序資料之深度學習分析框架

## 筆記本 01：資料獲取與預處理

**目標：**
1.  從「臺北市資料大平臺」下載所有月份的 [捷運 OD 流量資料](https://data.taipei/dataset/detail?id=63f31c7e-7fc3-418b-bd82-b95158755b4d)。
2.  將所有分散的 CSV 檔案合併成一個單一的 DataFrame。
3.  對合併後的資料進行初步的檢視與清理。

In [1]:
import os
import sys

# 將 src 目錄添加到 Python 的模組搜索路徑中
# 這樣我們才能導入自己寫的 data_loader 模組
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

print(f"專案根目錄: {project_root}")
print(f"原始碼目錄已加入 sys.path: {src_path}")

專案根目錄: /user_data/MRT_Traffic_Analysis
原始碼目錄已加入 sys.path: /user_data/MRT_Traffic_Analysis/src


## 步驟一：下載原始資料

此步驟的目標是透過 `src/data_loader.py` 中的 `DataLoader` 類別，自動從臺北市資料大平臺抓取所有月份的 OD 流量 CSV 檔案，並儲存至 `data/raw/` 目錄下。

In [2]:
# --- 設定正確的參數 ---
# 臺北捷運每日分時各站OD流量統計資料: https://data.taipei/dataset/detail?id=63f31c7e-7fc3-418b-bd82-b95158755b4d
DATASET_ID = 'eb481f58-1238-4cff-8caa-fa7bb20cb4f4' # API 的 ID 跟 網址上的 ID 會不一樣
RAW_DATA_DIR = os.path.join(project_root, 'data', 'raw')

In [3]:
# import os
# import sys

# # --- 再次確認環境設定 ---
# project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
# src_path = os.path.join(project_root, 'src')
# if src_path not in sys.path:
#     sys.path.append(src_path)
    
# from data_loader import DataLoader

# # --- 執行下載 ---
# print("=== 開始執行最終下載程序 ===")
# loader = DataLoader(dataset_id=DATASET_ID, limit=1000)
# loader.download_monthly_data(save_dir=RAW_DATA_DIR)

# # --- 驗證下載結果 ---
# print("\n--- 下載結果驗證 ---")
# try:
#     downloaded_files = os.listdir(RAW_DATA_DIR)
#     if downloaded_files:
#         print(f"在 '{RAW_DATA_DIR}' 目錄中找到 {len(downloaded_files)} 個檔案:")
#         # 列出前5個和後5個檔案作為範例
#         sorted_files = sorted(downloaded_files)
#         print("頭5個檔案:", sorted_files[:5])
#         print("末5個檔案:", sorted_files[-5:])
#     else:
#         print("目錄為空，請檢查下載過程是否有錯誤訊息。")
# except FileNotFoundError:
#     print(f"錯誤：找不到目錄 '{RAW_DATA_DIR}'。")

## 步驟二：讀取與合併資料
將所有月份的原始 CSV 檔合併成單一 DataFrame，並進行初步檢視。

In [ ]:
from data_processor import load_and_merge_raw_data
import pandas as pd

# 設定 pandas 顯示選項，以便更好地觀察 DataFrame
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

# --- 執行讀取與合併 ---
# RAW_DATA_DIR 變數我們在之前的儲存格已經定義過了
mrt_df = load_and_merge_raw_data(RAW_DATA_DIR)


# --- 初步檢視合併後的 DataFrame ---
if not mrt_df.empty:
    print("\n--- 合併後 DataFrame 的基本資訊 ---")
    mrt_df.info()
    
    print("\n\n--- 資料頭5筆 (Head) ---")
    display(mrt_df.head())
    
    print("\n\n--- 資料末5筆 (Tail) ---")
    display(mrt_df.tail())
    
    print(f"\n\n合併後的總筆數為: {len(mrt_df):,}")
else:
    print("DataFrame 是空的，請檢查前面的步驟。")

找到 100 個 CSV 檔案，準備讀取與合併...


正在讀取檔案: 100%|██████████| 100/100 [03:24<00:00,  2.05s/it]



正在合併所有資料...
所有資料已成功合併！

--- 合併後 DataFrame 的基本資訊 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828071570 entries, 0 to 828071569
Data columns (total 5 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   日期      object
 1   時段      int64 
 2   進站      object
 3   出站      object
 4   人次      int64 
dtypes: int64(2), object(3)
memory usage: 30.8+ GB


--- 資料頭5筆 (Head) ---


,日期,時段,進站,出站,人次
0,2017-01-01,0,松山機場,松山機場,0
1,2017-01-01,0,松山機場,中山國中,0
2,2017-01-01,0,松山機場,南京復興,0
3,2017-01-01,0,松山機場,忠孝復興,0
4,2017-01-01,0,松山機場,大安,0




--- 資料末5筆 (Tail) ---


,日期,時段,進站,出站,人次
828071565,2025-04-30,23,新北產業園區,徐匯中學,0
828071566,2025-04-30,23,新北產業園區,三和國中,0
828071567,2025-04-30,23,新北產業園區,三重國小,2
828071568,2025-04-30,23,新北產業園區,迴龍,3
828071569,2025-04-30,23,新北產業園區,丹鳳,1




合併後的總筆數為: 828,071,570


: 

## 步驟三：清理、轉換與彙總

將 8.28 億筆的原始資料進行以下處理：
1.  **建立時間戳**：將 `日期` 和 `時段` 欄位合併為標準的 `datetime` 格式。
2.  **優化資料型態**：將字串欄位轉換為 `category` 以大幅節省記憶體。
3.  **彙總資料**：將資料從「起訖站」層級，彙總到目標站點（臺北車站）的每小時進出站人次。

In [ ]:
from data_processor import process_and_aggregate_data

# --- 設定分析目標 ---
# 我們選擇 '臺北車站' 作為預測目標
TARGET_STATION = '臺北車站'

# --- 執行處理與彙總 ---
# mrt_df 是我們上一步合併好的巨大 DataFrame
if 'mrt_df' in locals() and not mrt_df.empty:
    station_df = process_and_aggregate_data(mrt_df, target_station=TARGET_STATION)

    # --- 檢視最終彙總的 DataFrame ---
    print("\n--- 最終彙總後 DataFrame 的基本資訊 ---")
    station_df.info()

    print("\n\n--- 彙總後資料頭5筆 (Head) ---")
    display(station_df.head())

    print("\n\n--- 彙總後資料末5筆 (Tail) ---")
    display(station_df.tail())
else:
    print("錯誤：找不到名為 'mrt_df' 的 DataFrame，請確認上一步已成功執行。")

=== 開始資料清理與轉換 ===
步驟 1/4: 建立標準時間戳...
